## Prototyping the construction of PySAL weights from geojson. Supports local files, uri, and serialized streams 

In [1]:
import geojson
import pysal as ps
import urllib2 as urllib

In [2]:
class PolygonCollection:
    def __init__(self, polygons, bbox=None):
        """

        Parameters
        ==========
        polygons: dict
                  key is polygon Id, value is PySAL Polygon object
        bbox: list (optional)
              [left, lower, right, upper]

        Notes
        =====
        bbox is supported in geojson specification at both the feature and feature collection level. However, not all GeoJSON writers generate the bbox at the feature collection level. 
        In those cases, the bbox property will be set on initial access.

        """
              
        self.type=ps.cg.Polygon
        self.n = len(polygons)
        self.polygons = polygons
        if bbox is None:
            self._bbox = None
        else:
            self._bbox = bbox
            
    @property
    def bbox(self):
        bboxes = np.array([self.polygons[p].bbox for p in self.polygons])
        mins = bboxes.min(axis=0)
        maxs = bboxes.max(axis=0)
        self._bbox = [ mins[0], mins[1], maxs[2], maxs[3] ]
        return self._bbox
        
    
    def __getitem__(self, index):
        return self.polygons[index]
            

In [3]:
def queen_geojson(gjobj):
    """
    Constructs a PySAL queen contiguity W from a GeoJSON feature collection consisting of Polygons and/or Multipolygons

    """
    first = gjobj['features'][0]['geometry']['type']
    if first == 'Polygon' or first =='MultiPolygon':
        polys = []
        ids = []
        i = 0
        for feature in gjobj['features']:
            if feature['geometry']['type'] == 'Polygon':
                polys.append(ps.cg.asShape(geojson.Polygon(feature['geometry']['coordinates'])))
            else:               
                polys.append(ps.cg.asShape(geojson.MultiPolygon(feature['geometry']['coordinates'])))
            ids.append(i)
            i += 1
        polygons = PolygonCollection(dict(zip(ids,polys)))
        neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(polygons).w
        return ps.W(neighbors)
    else:
        print "GeoJSON feature type must be 'Polygon' or 'MultiPolygon'"
        return None

In [4]:
def queen_from_geojson(uri):
    gjobj = geojson.load(urllib.urlopen(uri))
    return queen_geojson(gjobj)

In [5]:
def queen_from_geojsons(s):
    return queen_geojson(info)

In [6]:
def queen_from_geojsonf(fileName):
    fp = open(fileName)
    obj = geojson.load(fp)
    fp.close()
    return queen_geojson(obj)

In [7]:
# from a uri
uri = "http://toae.org/pub/columbus.json"

In [8]:
wq = queen_from_geojson(uri)

In [9]:
wq.n

49

In [10]:
# counties
# note that NAT.json needs to be created locally (too large for repos)
# "ogr2ogr -lco WRITE_BBOX=YES -f "GeoJSON" NAT.json NAT.shp
wf = queen_from_geojsonf(ps.examples.get_path("NAT.json"))

In [11]:
wf.n

3085

In [12]:
wq = ps.queen_from_shapefile(ps.examples.get_path("NAT.shp"))

In [13]:
wq.n

3085

In [14]:
wq.pct_nonzero

0.0019089598070866245

In [15]:
wf.pct_nonzero

0.0019089598070866245

In [16]:
for i in wq.neighbors:
    if set(wq.neighbors[i]) != set(wf.neighbors[i]):
        print wq.neighbors[i]
        print wf.neighbors[i]

In [17]:
uri = "http://toae.org/pub/NAT.json"
wi = queen_from_geojson(uri)

In [18]:
wi.n

3085